In [2]:
pip install xgboost


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   -- ------------------------------------- 2.9/56.8 MB 21.0 MB/s eta 0:00:03
   ----- ---------------------------------- 7.9/56.8 MB 19.4 MB/s eta 0:00:03
   -------- ------------------------------- 11.8/56.8 MB 18.9 MB/s eta 0:00:03
   ----------- ---------------------------- 15.7/56.8 MB 18.7 MB/s eta 0:00:03
   ------------- -------------------------- 19.7/56.8 MB 18.8 MB/s eta 0:00:02
   ---------------- ----------------------- 23.6/56.8 MB 18.7 MB/s eta 0:00:02
   ------------------- -------------------- 27.5/56.8 MB 18.6 MB/s eta 0:00:02
   ---------------------- ----------------- 31.5/56.8 MB 18.7 MB/s eta 0:00:02
   ------------------------ --------------- 35.1/56.8 MB 18.6 MB/s eta 0:00:02
   --------------------------- ------------ 39.1/56.8 MB 18.5 MB/s eta 0:00:01
   ------------------------------ --------- 43.3/56.8 MB 18.5 MB

In [88]:
# 📌 Step 1: Import Libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# 📌 Step 2: Load Dataset
data = pd.read_csv("final dataset  for kaggle.csv")

# Show first rows
data.head()


,CUSTOMER ID,CITY,STATE,SERVICE HISTORY,COMMON PROBLEM,SOLUTION USED,VEHICAL COMPANY
0,1,Ananthapuram,Andhra Pradesh,Oil Change; Brake Repair,Brake noise,Brake pad replacement,Maruti Suzuki
1,2,Ananthapuram,Andhra Pradesh,Engine Repair; Tire Rotation,Engine overheating,Radiator replacement,Hyundai
2,3,Ananthapuram,Andhra Pradesh,Bodywork; Paint Job,Dents and scratches,Body repair and repaint,Tata Motors
3,4,Ananthapuram,Andhra Pradesh,Oil Change; General Maintenance,Oil leak,Oil seal replacement,Mahindra & Mahindra
4,5,Ananthapuram,Andhra Pradesh,Tire Alignment; Engine Repair,Misaligned wheels,Wheel alignment,Honda


In [89]:
data.isna().sum()


CUSTOMER ID        0
CITY               0
STATE              0
SERVICE HISTORY    0
COMMON PROBLEM     0
SOLUTION USED      0
VEHICAL COMPANY    0
dtype: int64

In [90]:
data["SOLUTION USED"].unique()

array(['Brake pad replacement', 'Radiator replacement',
       'Body repair and repaint', 'Oil seal replacement',
       'Wheel alignment', 'Diagnostic scan and repairs',
       'Brake fluid replacement', 'Exhaust system repair',
       'Transmission fluid change', 'Body repair',
       'Spark plug replacement', 'Suspension adjustment',
       'Tire rotation and balancing', 'Transmission service',
       'Brake replacement', 'Diagnostic scan', 'Fuel system cleaning',
       'Fluid replacement', 'Cooling system flush', 'Battery replacement',
       'Tire alignment', 'Tire rotation',
       'Check coolant level; flush radiator; replace thermostat.',
       'Inspect pads/rotors; replace if worn.',
       ' Transmission fluid change', ' Fuel system cleaning',
       ' Exhaust system repair', ' Wheel alignment',
       ' Spark plug replacement', ' Suspension adjustment',
       ' Tire alignment', ' Battery replacement',
       ' Brake fluid replacement', ' Tire rotation and balancing',
    

In [91]:
# 📌 Step 3 (updated): Data Cleaning + Target Encoding
from sklearn.preprocessing import LabelEncoder

# Remove unwanted columns
cleaned_data = data.drop(columns=["CITY", "STATE"])

# Features and Target
X = cleaned_data.drop(columns=["SOLUTION USED"])
y = cleaned_data["SOLUTION USED"]

# Encode target labels into numbers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

cleaned_data.head()


,CUSTOMER ID,SERVICE HISTORY,COMMON PROBLEM,SOLUTION USED,VEHICAL COMPANY
0,1,Oil Change; Brake Repair,Brake noise,Brake pad replacement,Maruti Suzuki
1,2,Engine Repair; Tire Rotation,Engine overheating,Radiator replacement,Hyundai
2,3,Bodywork; Paint Job,Dents and scratches,Body repair and repaint,Tata Motors
3,4,Oil Change; General Maintenance,Oil leak,Oil seal replacement,Mahindra & Mahindra
4,5,Tire Alignment; Engine Repair,Misaligned wheels,Wheel alignment,Honda


In [92]:
y


0        Brake pad replacement
1         Radiator replacement
2      Body repair and repaint
3         Oil seal replacement
4              Wheel alignment
                ...           
503      Headlight restoration
504      Headlight restoration
505      Headlight restoration
506      Headlight restoration
507      Headlight restoration
Name: SOLUTION USED, Length: 508, dtype: object

In [93]:
# 📌 Step 4: Preprocessing (TF-IDF + OneHot + Pass-through)
preprocessor = ColumnTransformer(
    transformers=[
        ("service", TfidfVectorizer(), "SERVICE HISTORY"),
        ("problem", TfidfVectorizer(), "COMMON PROBLEM"),
        ("company", OneHotEncoder(handle_unknown="ignore"), ["VEHICAL COMPANY"])
    ],
    remainder="passthrough"  # keeps CUSTOMER ID
)


In [94]:
# 📌 Step 5: Define Models
models = {
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "Naive Bayes": MultinomialNB()
}


In [95]:
# 📌 Step 6: Train and Evaluate Models
results = {}
best_model = None
best_score = 0
best_name = ""


for name, model in models.items():
    pipeline = Pipeline([("preprocessor", preprocessor), ("classifier", model)])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    
    print(f"\n===== {name} =====")
    print("✅ Accuracy:", acc)
    print(classification_report(y_test, y_pred, zero_division=0))

    # 📌 Save best model
    if acc > best_score:
        best_score = acc
        best_model = pipeline
        best_name = name


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



===== Logistic Regression =====
✅ Accuracy: 0.8137254901960784
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         3
           6       0.82      1.00      0.90         9
           9       1.00      1.00      1.00         5
          10       0.50      1.00      0.67         3
          14       0.89      1.00      0.94         8
          15       0.67      1.00      0.80         2
          19       0.00      0.00      0.00         1
          20       0.92      1.00      0.96        23
          21       0.00      0.00      0.00         1
          22       0.00      0.00      0.00         1
          23       1.00      1.00      1.00         1
          26       0.71      0.83      0.77         6
          27       0.86      1.00      0.92         6
          30     

In [84]:
# 📌 Save the best model to a file
if best_model is not None:
    joblib.dump(best_model, f"{best_name}_best_model.pkl")
    print(f"\n🎉 Best model saved: {best_name}_best_model.pkl with accuracy {best_score}")


🎉 Best model saved: Random Forest_best_model.pkl with accuracy 0.8921568627450981


In [102]:
import joblib

# after training your pipeline
joblib.dump(pipeline, "best_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [97]:
# 📌 Step 7: Show Best Model
best_model = max(results, key=results.get)
print("🏆 Best Model is:", best_model, "with accuracy:", results[best_model])


🏆 Best Model is: Random Forest with accuracy: 0.8921568627450981


In [98]:
# 📌 Step 8: Create Prediction Function
def predict_solution(service_history, common_problem, company, customer_id=0):
    new_data = pd.DataFrame([{
        "CUSTOMER ID": customer_id,
        "SERVICE HISTORY": service_history,
        "COMMON PROBLEM": common_problem,
        "VEHICAL COMPANY": company
    }])
    
    # Use the best model
    final_pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", models[best_model])
    ])
    final_pipeline.fit(X, y)  # train on full dataset
    
    return final_pipeline.predict(new_data)[0]

# Example prediction
predict_solution("Oil Change; Brake Repair", "Brake noise", "Maruti Suzuki", customer_id=600)


' Brake pad replacement'

In [99]:
y_encoded

array([41, 52, 39, 51, 60, 46, 40, 47, 58, 38, 41, 53, 54, 57, 59, 42, 60,
       47, 45, 49, 48, 44, 58, 37, 55, 41, 53, 54, 57, 59, 42, 60, 47, 45,
       49, 48, 44, 58, 37, 56, 41, 54, 47, 58, 49, 45, 40, 60, 37, 53, 41,
       47, 58, 37, 48, 54, 42, 47, 43, 50, 33, 18, 14, 35, 26, 27, 29,  2,
       14,  5, 32,  9,  4, 15, 10,  7, 32, 14,  6, 10, 32, 23,  6, 26, 33,
       31,  5, 14,  2, 15,  9,  6, 30, 26, 27,  6, 15, 29, 14,  6,  6, 33,
       18, 14, 35, 26, 27, 29,  2, 14,  5, 32,  9,  4, 15, 10,  7, 32, 14,
        6, 10, 32, 23,  6, 26, 33, 31,  5, 14,  2, 15,  9,  6, 30, 26, 27,
        6, 15, 29, 14,  6,  6, 33, 18, 14, 35, 26, 27, 29,  2, 14,  5, 32,
        9,  4, 15, 10,  7, 32, 14,  6, 10, 32, 23,  6, 26, 33, 31,  5, 14,
        2, 15,  9,  6, 30, 26, 27,  6, 15, 29, 14,  6,  6, 33, 18, 14, 35,
       26, 27, 29,  2, 14,  5, 32,  9,  4, 15, 10,  7, 32, 14,  6, 10, 32,
       23,  6, 26, 33, 31,  5, 14,  2, 15,  9,  6, 30, 26, 27,  6, 15, 29,
       14,  6,  6, 33, 18

In [100]:
# Step 8: Train & Evaluate Models
for name, model in models.items():
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = {
        "accuracy": acc,
        "report": classification_report(y_test, y_pred, zero_division=0)
    }
    print(f"\n🔹 {name} Accuracy: {acc:.4f}")
    print(results[name]["report"])

    # Save best model
    if acc > best_accuracy:
        best_accuracy = acc
        best_model = pipeline


🔹 Logistic Regression Accuracy: 0.8137
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         3
           6       0.82      1.00      0.90         9
           9       1.00      1.00      1.00         5
          10       0.50      1.00      0.67         3
          14       0.89      1.00      0.94         8
          15       0.67      1.00      0.80         2
          19       0.00      0.00      0.00         1
          20       0.92      1.00      0.96        23
          21       0.00      0.00      0.00         1
          22       0.00      0.00      0.00         1
          23       1.00      1.00      1.00         1
          26       0.71      0.83      0.77         6
          27       0.86      1.00      0.92         6
          30       0.50      1.00      0.

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


NameError: name 'best_accuracy' is not defined

In [61]:
best_model

'Random Forest'

In [58]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [60]:
# Create KNN model
knn = KNeighborsClassifier(n_neighbors=3)  # k = 3 neighbors

# Train model
knn.fit(X_train, y_train)

# Predict
y_pred = knn.predict(X_test)

# Evaluate
print("Predictions:", y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))

Predictions: [15 29 20  3 14 20 14 14  5  4 15  9 14  2 27 20  4 41 42 37 35 20 20 25
  2 31 20 15  2 18 20 15 26 33  6 14  6 20 14 20 30 20 42  2 29 20  9 14
  7 18 18 20  6 10 44  5  2 20 29 47  6 41 15 20 59 20  9  5 20 20  2 41
 14 14 20 14 37 20 41 16 14  9  9  6  6  9 20 27 10 20  9 20 30 20 14  9
 35 20  6  5 10 27]
Accuracy: 0.30392156862745096


In [59]:
# Create KNN model
knn = KNeighborsRegressor(n_neighbors=2)

# Train model
knn.fit(X_train, y_train)

# Predict
y_pred = knn.predict(X_test)

# Evaluate
print("Predictions:", y_pred)
print("MSE:", mean_squared_error(y_test, y_pred))

NameError: name 'KNeighborsRegressor' is not defined

In [55]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import xgboost as xgb

# Step 2: Load dataset
df = pd.read_csv("final dataset  for kaggle.csv")

# Step 3: Add a synthetic 'Customer_ID' column
df.insert(0, "CUSTOMER ID ", range(1001, 1001 + len(df)))

# Step 4: Drop unwanted columns
unwanted = ["STATE", "CITY"]
df.drop(columns=[col for col in unwanted if col in df.columns], inplace=True)

# Step 5: Separate features (X) and target (y)
X = df.drop(columns=["SOLUTION USED"])   # Features
y = df["SOLUTION USED"]                  # Target

# Step 6: Encode categorical features
label_encoders = {}
for col in X.select_dtypes(include=["object"]).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Encode target
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(y)

# Step 7: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 8: Define XGBoost model
# Create XGBoost classifier
model = xgb.XGBClassifier(
    n_estimators=100,      # number of trees
    learning_rate=0.1,     # step size
    max_depth=3,           # depth of trees
    random_state=42,
    eval_metric="mlogloss" # evaluation metric
)

# Train model
model.fit(X_train, y_train)


# Step 9: Train model
xgb_model.fit(X_train, y_train)

# Step 10: Predictions
#y_pred = xgb_model.predict(X_test)

"""# Step 11: Evaluate
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=target_encoder.classes_))

# Confusion matrix
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d", cmap="Blues",
            xticklabels=target_encoder.classes_, yticklabels=target_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - XGBoost")
plt.show()

# Step 12: Feature importance
xgb.plot_importance(xgb_model, max_num_features=10, importance_type="gain")
plt.title("Top 10 Important Features")
plt.show()"""


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52], got [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 23 24 25 26 27
 28 29 30 31 32 33 34 35 37 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54
 55 57 58 59 60]

In [32]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
y = enc.fit_transform(cleaned_data[["SOLUTION USED"]]).astype(int).ravel()


In [35]:
# Fix encoding
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(cleaned_data["SOLUTION USED"].astype(str).str.strip())  # remove spaces

# Ensure no gaps
y = pd.Series(y).astype("int32").to_numpy()


In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [38]:
# Step 8: Define XGBoost model
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric="mlogloss"
)

# Step 9: Train model
xgb_model.fit(X_train, y_train)

# Step 10: Predictions
y_pred = xgb_model.predict(X_test)

# Step 11: Evaluate
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=target_encoder.classes_))


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37], got [ 0  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 27 28
 29 30 31 32 33 34 35 36 37 38 39 40 41 42]

In [63]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Load dataset
data = pd.read_csv("final dataset  for kaggle.csv")

# Drop unwanted columns
cleaned_data = data.drop(columns=["CITY", "STATE"])

# Features (X) and target (y)
X = cleaned_data.drop(columns=["SOLUTION USED"])
y = cleaned_data["SOLUTION USED"]

# Encode categorical features
label_encoders = {}
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Encode target variable
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(y)

# Train-test split (no stratify)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Evaluate
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 🔹 Save the model and encoders
joblib.dump(rf_model, "random_forest_model.pkl")
joblib.dump(label_encoders, "feature_encoders.pkl")
joblib.dump(target_encoder, "target_encoder.pkl")

print("✅ Model and encoders saved successfully!")


Random Forest Accuracy: 0.8627450980392157

Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.75      1.00      0.86         3
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         5
          10       1.00      1.00      1.00         3
          14       1.00      1.00      1.00         8
          15       0.67      1.00      0.80         2
          19       0.00      0.00      0.00         1
          20       0.96      1.00      0.98        23
          21       0.00      0.00      0.00         1
          22       0.00      0.00      0.00         1
          23       1.00      1.00      1.00         1
          25       0.00      0.00      0.00         0
          26       1.00      0.83      0.91         6
          27 

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklea

✅ Model and encoders saved successfully!


In [71]:
def predict_solution(service_history, common_problem, company, customer_id=999):
    # Create new dataframe
    new_data = pd.DataFrame([{
        "CUSTOMER ID": customer_id,
        "SERVICE HISTORY": service_history,
        "COMMON PROBLEM": common_problem,
        "VEHICAL COMPANY": company
    }])
    
    # Encode categorical columns using same encoders
    for col in new_data.columns:
        if col in label_encoders:  
            le = label_encoders[col]
            
            # Handle unseen labels
            new_data[col] = new_data[col].apply(
                lambda x: x if x in le.classes_ else "Unknown"
            )
            
            # Add "Unknown" to encoder if not present
            if "Unknown" not in le.classes_:
                le.classes_ = np.append(le.classes_, "Unknown")
            
            new_data[col] = le.transform(new_data[col])
    
    # Predict solution
    prediction = rf_model.predict(new_data)[0]
    
    # Decode prediction back to original solution label
    solution = target_encoder.inverse_transform([prediction])[0]
    
    return solution


# ✅ Example prediction
print("🔧 Required Tool / Solution:", 
      predict_solution("Oil Change; Brake Repair", "Brake noise", "Maruti Suzuki", customer_id=600))


🔧 Required Tool / Solution:  Headlight restoration


In [72]:
import joblib

# Save model
joblib.dump(rf_model, "rf_model.pkl")

# Save encoders (input + target encoder)
joblib.dump(label_encoders, "label_encoders.pkl")
joblib.dump(target_encoder, "target_encoder.pkl")


['target_encoder.pkl']

In [64]:
cleaned_data

,CUSTOMER ID,SERVICE HISTORY,COMMON PROBLEM,SOLUTION USED,VEHICAL COMPANY
0,1,Oil Change; Brake Repair,Brake noise,Brake pad replacement,Maruti Suzuki
1,2,Engine Repair; Tire Rotation,Engine overheating,Radiator replacement,Hyundai
2,3,Bodywork; Paint Job,Dents and scratches,Body repair and repaint,Tata Motors
3,4,Oil Change; General Maintenance,Oil leak,Oil seal replacement,Mahindra & Mahindra
4,5,Tire Alignment; Engine Repair,Misaligned wheels,Wheel alignment,Honda
...,...,...,...,...,...
503,496,Headlight Restoration; Suspension Check,Dull headlights,Headlight restoration,Hyundai Creta
504,497,Headlight Restoration; Tire Alignment,Cloudy headlights,Headlight restoration,Tata Nexon
505,498,Headlight Restoration; Oil Change,Cloudy headlights,Headlight restoration,Mahindra Scorpio
506,499,Headlight Restoration; Brake Check,Dim headlights,Headlight restoration,Nissan Kicks


In [65]:
X

,CUSTOMER ID,SERVICE HISTORY,COMMON PROBLEM,VEHICAL COMPANY
0,1,110,50,41
1,2,96,58,18
2,3,81,56,94
3,4,111,64,86
4,5,115,63,83
...,...,...,...,...
503,496,49,12,20
504,497,50,7,67
505,498,48,7,35
506,499,46,11,50


In [66]:
y

array([41, 52, 39, 51, 60, 46, 40, 47, 58, 38, 41, 53, 54, 57, 59, 42, 60,
       47, 45, 49, 48, 44, 58, 37, 55, 41, 53, 54, 57, 59, 42, 60, 47, 45,
       49, 48, 44, 58, 37, 56, 41, 54, 47, 58, 49, 45, 40, 60, 37, 53, 41,
       47, 58, 37, 48, 54, 42, 47, 43, 50, 33, 18, 14, 35, 26, 27, 29,  2,
       14,  5, 32,  9,  4, 15, 10,  7, 32, 14,  6, 10, 32, 23,  6, 26, 33,
       31,  5, 14,  2, 15,  9,  6, 30, 26, 27,  6, 15, 29, 14,  6,  6, 33,
       18, 14, 35, 26, 27, 29,  2, 14,  5, 32,  9,  4, 15, 10,  7, 32, 14,
        6, 10, 32, 23,  6, 26, 33, 31,  5, 14,  2, 15,  9,  6, 30, 26, 27,
        6, 15, 29, 14,  6,  6, 33, 18, 14, 35, 26, 27, 29,  2, 14,  5, 32,
        9,  4, 15, 10,  7, 32, 14,  6, 10, 32, 23,  6, 26, 33, 31,  5, 14,
        2, 15,  9,  6, 30, 26, 27,  6, 15, 29, 14,  6,  6, 33, 18, 14, 35,
       26, 27, 29,  2, 14,  5, 32,  9,  4, 15, 10,  7, 32, 14,  6, 10, 32,
       23,  6, 26, 33, 31,  5, 14,  2, 15,  9,  6, 30, 26, 27,  6, 15, 29,
       14,  6,  6, 33, 18

In [69]:
# Load saved model and encoders
rf_model = joblib.load("random_forest_model.pkl")
label_encoders = joblib.load("feature_encoders.pkl")
target_encoder = joblib.load("target_encoder.pkl")

# Example new input (replace values with actual part details)
new_data = pd.DataFrame([{
    "CUSTOMER ID": 12345,
    "SERVICE HISTORY": "Headlight Restoration; Oil Change",
    "COMMON PROBLEM": "Brake noise",
    "VEHICAL COMPANY": "Toyota"
}])

# Apply label encoding to features
for col in new_data.select_dtypes(include="object").columns:
    if col in label_encoders:
        new_data[col] = label_encoders[col].transform(new_data[col].astype(str))

# Predict solution
prediction = rf_model.predict(new_data)
solution = target_encoder.inverse_transform(prediction)

print("🔧 Required Tool / Solution:", solution[0])


ValueError: y contains previously unseen labels: 'Headlight Restoration; Oil Change'